In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='1'
import time
import numpy as np
import tensorflow as tf
from VGG16_GAP import VGG16_GAP
from VGG16_flatten import VGG16_flatten

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import numpy as np
import pandas as pd
import skimage.io as imageio
import pickle

In [3]:
from progress.bar import Bar
from ipywidgets import IntProgress
from IPython.display import display

In [4]:
with open('save/label_dict.pkl', 'rb') as f:
    y_dict = pickle.load(f)

In [5]:
HOME_DIR = "/home/cmchang/DLCV2018SPRING/final/"
TRAIN_DIR = HOME_DIR+"dlcv_final_2_dataset/train/"
VALID_DIR = HOME_DIR+"dlcv_final_2_dataset/val/"

In [6]:
dtrain = pd.read_csv(HOME_DIR+"dlcv_final_2_dataset/train_id.txt", header=None,sep=" ", names=["img", "id"])
dvalid = pd.read_csv(HOME_DIR+"dlcv_final_2_dataset/val_id.txt", header=None,sep=" ", names=["img", "id"])

In [7]:
train_list = list(TRAIN_DIR+dtrain.img)
valid_list = list(VALID_DIR+dvalid.img)

In [8]:
def readImgList(file_list):
    images = list()
    for i, file in enumerate(file_list):
        print(i, end="\r")
        img = imageio.imread(file)
        img = img.astype(int)
        images.append(img)
    return np.array(images)

In [9]:
def transformLabel(id_list, y_dict):
    label = list()
    for uid in list(id_list):
        label.append(y_dict[uid])
    return np.array(label)

In [10]:
def one_hot_encoding(class_numbers, num_classes):
    return np.eye(num_classes, dtype=float)[class_numbers]

In [11]:
def initialize_uninitialized(sess):
    global_vars = tf.global_variables()
    is_not_initialized = sess.run([tf.is_variable_initialized(var) for var in global_vars])
    not_initialized_vars = [v for (v,f) in zip(global_vars, is_not_initialized) if not f]
    if len(not_initialized_vars): 
            sess.run(tf.variables_initializer(not_initialized_vars))

In [12]:
Xtrain = readImgList(train_list)

In [13]:
Xvalid = readImgList(valid_list)

In [14]:
Xtrain.shape

(56475, 218, 178, 3)

In [15]:
ytrain = transformLabel(list(dtrain.id), y_dict)

In [16]:
yvalid = transformLabel(list(dvalid.id), y_dict)

In [17]:
Ytrain = one_hot_encoding(ytrain, len(y_dict))
Yvalid = one_hot_encoding(yvalid, len(y_dict))

In [18]:
scope_name = "Model2"

In [19]:
model = VGG16_GAP(scope_name=scope_name)

In [20]:
FLAG_init_from = HOME_DIR+"finetune_gap_v3_rescale0-1_save_linear/sparse_dict.npy"
FLAG_prof_type = "linear"
FLAG_lambda_s = 4e-3
FLAG_lambda_m = 2e-5
FLAG_decay = 1e-5
FLAG_lr = 1e-4
FLAG_keep_prob = 1.0
FLAG_save_dir = HOME_DIR+"dynamic_gap_v3_rescale0-1_save_linear/"

In [21]:
model.build(vgg16_npy_path=FLAG_init_from,
            shape=Xtrain.shape[1:],
            classes=len(y_dict),
            prof_type=FLAG_prof_type,
            conv_pre_training=True,
            fc_pre_training=True,
            new_bn=False)

build model started
npy file loaded
build model finished: 0s


In [22]:
dp = [1.0, 0.5]
tasks = [str(int(p*100)) for p in dp]
model.set_idp_operation(dp=dp, decay=FLAG_decay)

DP under test: [1.  0.5]
[None, 218, 178, 3]
[None, 218, 178, 29]
AFTER [None, 218, 178, 29]
[None, 109, 89, 32]
AFTER [None, 109, 89, 32]
[None, 109, 89, 39]
AFTER [None, 109, 89, 39]
[None, 55, 45, 66]
AFTER [None, 55, 45, 66]
[None, 55, 45, 79]
AFTER [None, 55, 45, 79]
[None, 55, 45, 99]
AFTER [None, 55, 45, 99]
[None, 28, 23, 85]
AFTER [None, 28, 23, 85]
[None, 28, 23, 88]
AFTER [None, 28, 23, 88]
[None, 28, 23, 80]
AFTER [None, 28, 23, 80]
[None, 14, 12, 100]
AFTER [None, 14, 12, 100]
[None, 14, 12, 85]
AFTER [None, 14, 12, 85]
[None, 14, 12, 108]
AFTER [None, 14, 12, 108]
[None, 218, 178, 3]
[None, 218, 178, 29]
AFTER [None, 218, 178, 14]
[None, 109, 89, 32]
AFTER [None, 109, 89, 16]
[None, 109, 89, 39]
AFTER [None, 109, 89, 19]
[None, 55, 45, 66]
AFTER [None, 55, 45, 33]
[None, 55, 45, 79]
AFTER [None, 55, 45, 39]
[None, 55, 45, 99]
AFTER [None, 55, 45, 49]
[None, 28, 23, 85]
AFTER [None, 28, 23, 42]
[None, 28, 23, 88]
AFTER [None, 28, 23, 44]
[None, 28, 23, 80]
AFTER [None, 28,

In [23]:
obj = 0.0
for cur_task in tasks:
    print(cur_task)
    obj += model.loss_dict[cur_task]

100
50


In [24]:
import imgaug as ia
from imgaug import augmenters as iaa
sometimes = lambda aug: iaa.Sometimes(0.5, aug)
transform = iaa.Sequential([
    sometimes(iaa.Affine(translate_percent={"x": (-0.15, 0.15), "y": (-0.15, 0.15)})),
    sometimes(iaa.Affine(scale={"x": (0.85, 1.15), "y":(0.85, 1.15)})),
    sometimes(iaa.Affine(rotate=(-45, 45))),
    sometimes(iaa.Fliplr(0.5))
])

In [ ]:
print("===== create directory =====")
if not os.path.exists(FLAG_save_dir):
    os.makedirs(FLAG_save_dir)

arr_spareness = []

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    augment = False

   # hyper parameters
    batch_size = 32
    epoch = 100
    early_stop_patience = 10
    min_delta = 0.0001
    opt_type = 'adam'

    # recorder
    epoch_counter = 0
    history = list()

    # optimizer
    global_step = tf.Variable(0, trainable=False)

    # Passing global_step to minimize() will increment it at each step.
    learning_rate = FLAG_lr
    opt = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.5)

    checkpoint_path = os.path.join(FLAG_save_dir, 'model.ckpt')
    
    # trainable variables
    train_vars = list()
    for var in tf.trainable_variables():
        if model.scope_name in var.name:
            train_vars.append(var)

    for rm in model.gamma_var:
        train_vars.remove(rm)
        print('%s is not trainable.'% rm)
    
    for var in train_vars:
        if '_mean' in var.name:
            train_vars.remove(var)
            print('%s is not trainable.'% var)
    
    for var in train_vars:
        if '_beta' in var.name:
            train_vars.remove(var)
            print('%s is not trainable.'% var)
    
    for var in train_vars:
        if '_variance' in var.name:
            train_vars.remove(var)
            print('%s is not trainable.'% var)
    
    print(train_vars)
            
    train_op = opt.minimize(obj, global_step=global_step, var_list=train_vars)
    
    saver = tf.train.Saver(tf.global_variables(), max_to_keep=len(tasks))

    # progress bar
    ptrain = IntProgress()
    pval = IntProgress()
    display(ptrain)
    display(pval)
    ptrain.max = int(Xtrain.shape[0]/batch_size)
    pval.max = int(Xvalid.shape[0]/batch_size)

    spareness = model.spareness(thresh=0.05)
    print("initial spareness: %s" % sess.run(spareness))

    # re-initialize
    initialize_uninitialized(sess)

    # reset due to adding a new task
    patience_counter = 0
    current_best_val_accu = 0

    # optimize when the aggregated obj
    while(patience_counter < early_stop_patience and epoch_counter < epoch):
        
        # start training
        stime = time.time()
        bar_train = Bar('Training', max=int(Xtrain.shape[0]/batch_size), suffix='%(index)d/%(max)d - %(percent).1f%% - %(eta)ds')
        bar_val =  Bar('Validation', max=int(Xvalid.shape[0]/batch_size), suffix='%(index)d/%(max)d - %(percent).1f%% - %(eta)ds')
        train_loss, train_accu = 0.0, 0.0
        
        if augment:
            def load_batches():
                for i in range(int(Xtrain.shape[0]/batch_size)):
                    st = i*batch_size
                    ed = (i+1)*batch_size
                    batch = ia.Batch(images=Xtrain[st:ed,:,:,:], data=Ytrain[st:ed,:])
                    yield batch

            batch_loader = ia.BatchLoader(load_batches)
            bg_augmenter = ia.BackgroundAugmenter(batch_loader=batch_loader, augseq=transform, nb_workers=1)

            while True:
                batch = bg_augmenter.get_batch()
                if batch is None:
                    print("Finished epoch.")
                    break
                x_images_aug = batch.images_aug
                y_images = batch.data
                loss, accu, _ = sess.run([obj, model.accu_dict[cur_task], train_op], feed_dict={model.x: x_images_aug,
                                model.y: y_images,
                                model.is_train: True})
                bar_train.next()
                train_loss += loss
                train_accu += accu
                ptrain.value +=1
                ptrain.description = "Training %s/%s" % (ptrain.value, ptrain.max)
            batch_loader.terminate()
            bg_augmenter.terminate()
        else:
            for i in range(int(Xtrain.shape[0]/batch_size)):
                st = i*batch_size
                ed = (i+1)*batch_size
                loss, accu, _ = sess.run([obj, model.accu_dict[tasks[0]], train_op],
                                                    feed_dict={model.x: Xtrain[st:ed,:],
                                                               model.y: Ytrain[st:ed,:],
                                                               model.is_train: False})
                train_loss += loss
                train_accu += accu
                ptrain.value +=1
                ptrain.description = "Training %s/%s" % (ptrain.value, ptrain.max)
        
        train_loss = train_loss/ptrain.value
        train_accu = train_accu/ptrain.value


        # validation
        val_loss = 0
        val_accu1 = 0
        val_accu2 = 0
        for i in range(int(Xvalid.shape[0]/batch_size)):
            st = i*batch_size
            ed = (i+1)*batch_size
            loss, accu1, accu2 = sess.run([obj, model.accu_dict[tasks[0]], model.accu_dict[tasks[-1]]],
                                                feed_dict={model.x: Xvalid[st:ed,:],
                                                           model.y: Yvalid[st:ed,:],
                                                           model.is_train: False})
            val_loss += loss
            val_accu1 += accu1
            val_accu2 += accu2
            pval.value += 1
            pval.description = "Testing %s/%s" % (pval.value, pval.value)
        val_loss = val_loss/pval.value
        val_accu1 = val_accu1/pval.value
        val_accu2 = val_accu2/pval.value
        val_accu = (val_accu1+val_accu2)/2
        
        print("\nspareness: %s" % sess.run(spareness))
        # early stopping check
        if (val_accu - current_best_val_accu) > min_delta:
            current_best_val_accu = val_accu
            patience_counter = 0

            para_dict = sess.run(model.para_dict)
            np.save(os.path.join(FLAG_save_dir, "para_dict.npy"), para_dict)
            print("save in %s" % os.path.join(FLAG_save_dir, "para_dict.npy"))
        else:
            patience_counter += 1

        # shuffle Xtrain and Ytrain in the next epoch
        idx = np.random.permutation(Xtrain.shape[0])
        Xtrain, Ytrain = Xtrain[idx,:,:,:], Ytrain[idx,:]

        # epoch end
        # writer.add_summary(epoch_summary, epoch_counter)
        epoch_counter += 1

        ptrain.value = 0
        pval.value = 0
        bar_train.finish()
        bar_val.finish()

        print("Epoch %s (%s), %s sec >> train loss: %.4f, train accu: %.4f, val loss: %.4f, val accu at %s: %.4f, val accu at %s: %.4f" % (epoch_counter, patience_counter, round(time.time()-stime,2), train_loss, train_accu, val_loss, tasks[0], val_accu1, tasks[-1], val_accu2))
        history.append([train_loss, train_accu, val_loss, val_accu ])
        
        if epoch_counter % 10 == 0:
            import matplotlib.pyplot as plt
            df = pd.DataFrame(history)
            df.columns = ['train_loss', 'train_accu', 'val_loss', 'val_accu']
            df[['train_loss', 'val_loss']].plot()
            plt.savefig(os.path.join(FLAG_save_dir, 'loss.png'))
            df[['train_accu', 'val_accu']].plot()
            plt.savefig(os.path.join(FLAG_save_dir, 'accu.png'))
            
    saver.save(sess, checkpoint_path, global_step=epoch_counter)

===== create directory =====
<tf.Variable 'Model2/conv1_1_gamma:0' shape=(29,) dtype=float32_ref> is not trainable.
<tf.Variable 'Model2/conv1_2_gamma:0' shape=(32,) dtype=float32_ref> is not trainable.
<tf.Variable 'Model2/conv2_1_gamma:0' shape=(39,) dtype=float32_ref> is not trainable.
<tf.Variable 'Model2/conv2_2_gamma:0' shape=(66,) dtype=float32_ref> is not trainable.
<tf.Variable 'Model2/conv3_1_gamma:0' shape=(79,) dtype=float32_ref> is not trainable.
<tf.Variable 'Model2/conv3_2_gamma:0' shape=(99,) dtype=float32_ref> is not trainable.
<tf.Variable 'Model2/conv3_3_gamma:0' shape=(85,) dtype=float32_ref> is not trainable.
<tf.Variable 'Model2/conv4_1_gamma:0' shape=(88,) dtype=float32_ref> is not trainable.
<tf.Variable 'Model2/conv4_2_gamma:0' shape=(80,) dtype=float32_ref> is not trainable.
<tf.Variable 'Model2/conv4_3_gamma:0' shape=(100,) dtype=float32_ref> is not trainable.
<tf.Variable 'Model2/conv5_1_gamma:0' shape=(85,) dtype=float32_ref> is not trainable.
<tf.Variable 

IntProgress(value=0)

IntProgress(value=0)

initial spareness: 0.0

spareness: 0.0
save in /home/cmchang/DLCV2018SPRING/final/dynamic_gap_v3_rescale0-1_save_linear/para_dict.npy
Epoch 1 (0), 475.36 sec >> train loss: 14.0934, train accu: 0.0820, val loss: 11.8437, val accu at 100: 0.2526, val accu at 50: 0.0003

spareness: 0.0
save in /home/cmchang/DLCV2018SPRING/final/dynamic_gap_v3_rescale0-1_save_linear/para_dict.npy
Epoch 2 (0), 472.22 sec >> train loss: 10.6258, train accu: 0.4138, val loss: 10.6546, val accu at 100: 0.4381, val accu at 50: 0.0003

spareness: 0.0
save in /home/cmchang/DLCV2018SPRING/final/dynamic_gap_v3_rescale0-1_save_linear/para_dict.npy
Epoch 3 (0), 472.51 sec >> train loss: 9.6416, train accu: 0.5762, val loss: 10.0296, val accu at 100: 0.5312, val accu at 50: 0.0011

spareness: 0.0
save in /home/cmchang/DLCV2018SPRING/final/dynamic_gap_v3_rescale0-1_save_linear/para_dict.npy
Epoch 4 (0), 473.02 sec >> train loss: 9.0392, train accu: 0.6548, val loss: 9.6569, val accu at 100: 0.5683, val accu at 50: 0.0


spareness: 0.0
save in /home/cmchang/DLCV2018SPRING/final/dynamic_gap_v3_rescale0-1_save_linear/para_dict.npy
Epoch 39 (0), 471.49 sec >> train loss: 2.4026, train accu: 0.9621, val loss: 5.8466, val accu at 100: 0.6833, val accu at 50: 0.4336

spareness: 0.0
save in /home/cmchang/DLCV2018SPRING/final/dynamic_gap_v3_rescale0-1_save_linear/para_dict.npy
Epoch 40 (0), 473.94 sec >> train loss: 2.3350, train accu: 0.9622, val loss: 5.7228, val accu at 100: 0.6906, val accu at 50: 0.4457

spareness: 0.0
Epoch 41 (1), 471.18 sec >> train loss: 2.2709, train accu: 0.9638, val loss: 5.7588, val accu at 100: 0.6781, val accu at 50: 0.4432

spareness: 0.0
Epoch 42 (2), 471.49 sec >> train loss: 2.2127, train accu: 0.9651, val loss: 6.0077, val accu at 100: 0.6840, val accu at 50: 0.3974

spareness: 0.0
Epoch 43 (3), 471.66 sec >> train loss: 2.1572, train accu: 0.9648, val loss: 6.0978, val accu at 100: 0.6833, val accu at 50: 0.4256

spareness: 0.0
Epoch 44 (4), 473.49 sec >> train loss: 2.09